In [2]:
# 서버 구동

from flask import Flask, send_file, request
import import_ipynb
from gqa_module import *
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%env OPENAI_API_KEY=<Enter you key here>
from engine.utils import ProgramGenerator, ProgramInterpreter
from prompts.imgeEdit import PROMPT

app = Flask(__name__)

interpreter = ProgramInterpreter(dataset='imageEdit')

def create_prompt(instruction):
    return PROMPT.format(instruction=instruction)

generator = ProgramGenerator(prompter=create_prompt)

@app.route('/')
def index():
    return f'''<!doctype html>
    <html>
        <body>
            <h1><a href="/">visprog</a></h1>
            <ol>
                <a href="/imageEditResult">실행</a>
            </ol>
        </body>
    </html>
    '''

@app.route('/imageEditResult')
def visResult():
    
    #url = 'assets/bollywood.png'
    #chat = "Create a colorpop of the man in black henley and also blur the background"
    
    url = 'assets/camel2.png'
    chat = "Select the camel, and blur the background"
    result = exe_imageEdit(url, chat, interpreter, generator)
    
    result_path = 'result/final1.jpg'
    result.save(result_path)
    
    return f'''<!doctype html>
    <html>
        <body>
            <h1><a href="/">visprog</a></h1>
            <div> instruction : {chat}<div>
            <hr>원본 : <img src='/get_image?url={url}' width='200' height='200'></hr>
            <hr>수정 : <img src='/get_image?url={result_path}' width='200' height='200'></hr>
        </body>
    </html>
    '''

@app.route('/get_image')
def get_image():
    # URL로 전달된 파일 경로 가져오기
    url = request.args.get('url')
    
    # 파일 경로로부터 이미지 파일 읽어오기
    img_path = os.path.join(app.root_path, url)
    
    # 이미지 파일을 클라이언트에게 전송
    return send_file(img_path)

app.run()